<a href="https://colab.research.google.com/github/its-areejio/LevelUpLboro/blob/main/menu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LevelUpLboro | Games Store Booking System



Last Updated: 29/12/2025 | Author: F525046

Welcome to my coursework submission for 25COA122: a games booking system created using Python, ipywidgets and matplotlib. The project is split into teach of the following functionalities:
- **Database Validation**: Consisting of 4 functions used to ensure that all processes are allowed in the system, whether this be: validating a customer's subscription; checking a game's availability; checking if a customer has reached their rental limit;and ensuring that no booked sessions go above the 50-person in-store limit,
- **Find a Game**: Performs a search of either board or video games in the stores inventory, displaying it's stored information,
- **Rent a Game**: An extension of the findGame display, allows customers to rent games provided that they aren't currently being rented out elsewhere (the assumption is made that only one copy of each game is owned),
- **Return a Game**: Allows customers to return games and give related feedback alongside a numerical rating,
- **Book a Session**: Allows customers to book either an afternoon or evening session on a given day, provided their chosen session isn't full,
- **Inventory Management**: Shows how many rentals occur at different times in the year, allowing management to identify booms and when marketing is needed more. Also displays the genre-split of voices, with the relevant function also returning the average rating of each genre though displaying this on-hover is a future implememtation.

In order to run the project, click the 'Run all' button at the top of the screen, upload the files found in the same folder as this menu and scroll to the 'Main Program' section at the bottom of the notebook to see the GUI and interact with the above functions. Enjoy using LevelUpLboro!

In [ ]:
'''
Upload files associated with the project from the zipped folder
'''

from google.colab import files
uploaded = files.upload()

##Database Validation

In [ ]:
'''
Validating a given customerID using imported pyc subscriptionManager
'''

import subscriptionManager
import datetime

def checkSubscription(customerID):
  subscriptions = subscriptionManager.load_subscriptions()
  if customerID in subscriptions and subscriptions[customerID]['EndDate'] > datetime.datetime.today():
    valid = True
    subscriptionType = subscriptions[customerID]['SubscriptionType']
    return valid, subscriptionType
  else:
    return False

In [ ]:
'''
Checking against list of active rentals (new file created) to see if a game can
be rented and if a customer is eligible to rent a game
ActiveRentals is stored in the form:
gameID (current[0]),RentalDate, DueDate, CustomerID (current[3])
'''

def checkRental(gameID):
  available = True
  with open('ActiveRentals.txt', 'r') as ActiveRentals:
    for line in ActiveRentals:
      current = line.strip().split(',')
      if len(current) >= 1 and current[0].strip() == gameID:
        available = False
        break
  return available

def checkCustomerRentals(customerID):
  rentalCount = 0
  subscriptionStatus = checkSubscription(customerID)
  rentalLimit = 0

  if isinstance(subscriptionStatus, tuple):
    isValid = subscriptionStatus[0]
    subscriptionType = subscriptionStatus[1]
    if isValid:
      if subscriptionType == 'Basic':
        rentalLimit = 4
      elif subscriptionType == 'Premium':
        rentalLimit = 10

  with open('ActiveRentals.txt', 'r') as ActiveRentals:
    for line in ActiveRentals:
      current = line.strip().split(',')
      if len(current) >= 4 and current[3].strip() == customerID:
        rentalCount += 1
  return not (rentalCount < rentalLimit)

In [ ]:
'''
Checking against current bookings to see if a maximum capacity of 50 people has
been met for a given slot
'''

def checkCapacity(date, timeSlot, guests):
  capacity = guests
  atCapacity = False
  with open('Bookings.txt', 'r') as bookings:
    lines = bookings.readlines()
    for line in lines[1:]:
      line = line.strip()
      if not line:
          continue
      current = line.split(',')
      if len(current) >= 3:
        if current[1].strip() == str(date):
          if timeSlot == 'afternoon' and current[2].strip() == '2pm':
            capacity += 1
          elif timeSlot == 'evening' and current[2].strip() == '6pm':
            capacity += 1
  if capacity > 50:
    atCapacity = True
  return atCapacity

## Find a Game

In [ ]:
from ipywidgets import *
from IPython.display import *

'''
The findGame function takes the inputted gameID and checks for it against the
relevant text file depending on if it's a board or video game. There are three
possible states for the game to be in:
- Owned but being rented out
- Owned and not being rented out
- Not part of the store's inventory
Each of the above cases is handed accordingly by the function, with labels
displayed within the relevant 'Accordion' in the original display to avoid
confusion
'''

def findGame(button, findGameDisplay, searchDisplay, gameType, gameID):
  gameInfo = None
  gameOwned = False
  if gameType == 'board':
    with open('BoardGameInfo.txt', 'r') as BoardGameInfo:
      for line in BoardGameInfo:
        current = line.split(',')
        if current[0].strip() == gameID.strip():
          gameInfo = line.strip()
          gameOwned = True
          break
  elif gameType == 'video':
    with open('VideoGameInfo.txt', 'r') as VideoGameInfo:
      for line in VideoGameInfo:
        current = line.split(',')
        if current[0].strip() == gameID.strip():
          gameInfo = line.strip()
          gameOwned = True
          break
  #Ensuring that labels added to the display are in the correct 'Accordion'
  currentChildren = list(findGameDisplay.children)
  if gameOwned:
    available = checkRental(gameID)
    if available: #Game is owned and not being rented out
      rentButton = Button(description='Rent')
      rentButton.on_click(lambda b: rentDisplay(b, findGameDisplay, searchDisplay, gameType, gameID))
      result = HBox([Label(value=gameInfo), rentButton])
      extendedDisplay = VBox([searchDisplay, result])
      if gameType == 'board':
        currentChildren[0] = extendedDisplay
      elif gameType == 'video':
        currentChildren[1] = extendedDisplay
    else: #Game is owned and being rented out
      unavailableButton = Button(description='Unavailable')
      result = HBox([Label(value=gameInfo), unavailableButton])
      extendedDisplay = VBox([searchDisplay, result])
      if gameType == 'board':
        currentChildren[0] = extendedDisplay
      elif gameType == 'video':
        currentChildren[1] = extendedDisplay
    findGameDisplay.children = tuple(currentChildren)
  else: #Game not in inventory
    extendedDisplay = VBox([searchDisplay, Label(value='Game not found.')])
    if gameType == 'board':
      currentChildren[0] = extendedDisplay
    elif gameType == 'video':
      currentChildren[1] = extendedDisplay
    findGameDisplay.children = tuple(currentChildren)

## Rent a Game

In [ ]:
from ipywidgets import *
from IPython.display import clear_output
from dateutil.relativedelta import relativedelta

'''
Renting functionality has been split into two functions:
- rentDisplay which provides a form for the manager to fill in with the customer's
ID as well as displaying the selected game to rent out
- rentGame which checks if the rental is valid for a specific customer using the
checkSubscription function. A customer needs to both have a valid, active
subscription while also not renting out more than their alloted games
(Basic = 4, Premium = 10). This also allocates how long the game is rented out
for (Basic = 2 weeks, Premium = 5 weeks)
'''

def rentGame(gameID, customerCheck, findGameDisplay, extendedDisplay, gameType):
  customerID = customerCheck.value
  subscription = checkSubscription(customerID)
  currentChildren = list(findGameDisplay.children)

  if isinstance(subscription, tuple):
    hitLimit = checkCustomerRentals(customerID)
    if not hitLimit:
      #To list of all rental history, add this with no return date
      currentDate = datetime.date.today()
      with open('Rental.txt', 'a') as rental:
        rental.write(f'\n{gameID},{currentDate}, ,{customerID}')
      #To list of active rentals, add this with due date
      subscriptionType = subscription[1]
      if subscriptionType == 'Basic':
        dueDate = currentDate + relativedelta(weeks=2)
      else:
        dueDate = currentDate + relativedelta(weeks=5)
      with open('ActiveRentals.txt', 'a') as ActiveRentals:
        ActiveRentals.write(f'\n{gameID},{currentDate},{dueDate},{customerID}')
      extendedDisplay = VBox([extendedDisplay, Label(value='Rental successful.')])
    else:
      extendedDisplay = VBox([extendedDisplay, Label(value=f'{customerID} has too many active rentals.')])
  else:
    extendedDisplay = VBox([extendedDisplay, Label(value='No such customer.')])

  if gameType == 'board':
    currentChildren[0] = extendedDisplay
  elif gameType == 'video':
    currentChildren[1] = extendedDisplay
  findGameDisplay.children = tuple(currentChildren)

#New display to overlay FindGame Tab
def rentDisplay(button, findGameDisplay, searchDisplay, gameType, gameID):
  rentLabel = Label(value = f'Rent:{gameID}')
  customerCheck = Text(
    description = 'CustomerID',
    disabled = False
  )
  rent = Button(description = 'Rent Game')
  rent.on_click(lambda b: rentGame(gameID, customerCheck, findGameDisplay, extendedDisplay, gameType))
  rentScreen = VBox([rentLabel, customerCheck, rent])
  extendedDisplay = VBox([searchDisplay, rentScreen])
  currentChildren = list(findGameDisplay.children)
  if gameType == 'board':
    currentChildren[0] = extendedDisplay
  elif gameType == 'video':
    currentChildren[1] = extendedDisplay
  findGameDisplay.children = tuple(currentChildren)

## Return a Game

In [ ]:
import feedbackManager
import datetime

''''
To return a game, the details entered in the form are compared against the
ActiveRentals file to make sure that only games being rented out from the
store are being returned. This entry is then deleted to indicate the return is
complete. The Rental file shows all rentals made and is updated with the date of
this return (before now holding an empty string)
'''

def returnGame(button, tab, returnDisplay, gameID, rating, feedback):
  if not checkRental(gameID):
    with open('ActiveRentals.txt', 'r') as ActiveRentals:
      activeRentalLines = ActiveRentals.readlines()
    newActiveRentalLines = []
    for line in activeRentalLines:
      current = line.strip().split(',')
      if len(current) >= 1 and current[0].strip() != gameID:
        newActiveRentalLines.append(line)
    with open('ActiveRentals.txt', 'w') as ActiveRentals:
      ActiveRentals.writelines(newActiveRentalLines)

    with open('Rental.txt', 'r') as Rental:
      rentalHistoryLines = Rental.readlines()
    newRentalHistoryLines = []
    foundRentalToUpdate = False
    for line in rentalHistoryLines:
      current = line.strip().split(',')
      if len(current) >= 4 and current[0].strip() == gameID and current[2].strip() == '' and not foundRentalToUpdate:
        newRentalHistoryLines.append(f'{current[0]},{current[1]},{datetime.date.today()},{current[3]}\n')
        foundRentalToUpdate = True
      else:
        newRentalHistoryLines.append(line)
    with open('Rental.txt', 'w') as Rental:
      Rental.writelines(newRentalHistoryLines)

    feedbackManager.add_feedback(gameID, rating, feedback, 'Game_Feedback.txt')
    extendedDisplay = VBox([returnDisplay, Label(value='Return complete')])

  else:
    extendedDisplay = VBox([returnDisplay, Label(value='Error: No such rental')])

  currentChildren = list(tab.children)
  currentChildren[1] = extendedDisplay
  tab.children = tuple(currentChildren)

## Book a Session

In [ ]:
from ipywidgets import *
from IPython.display import clear_output

'''
Customers have the option of booking a session in either the afternoon or evening
of any day (as the opening days of the store is unknown, this is kept open). This
function handles the validation of information entered in the booking form
to ensure that:
- The customer is a valid one
- The store is not at maximum capacity for the desired session
A label is added ot the relevant 'Accordion' just as in the FindGame Tab
'''

def bookSession(button, bookSessionDisplay, sessionDisplay, timeSlot, date, customerID, guests):
  currentChildren = list(bookSessionDisplay.children)
  validCustomer = checkSubscription(customerID)
  atCapacity = checkCapacity(date, timeSlot, guests)
  if validCustomer and not atCapacity:
    with open('Bookings.txt', 'a') as bookings:
      if timeSlot == 'afternoon':
        bookings.write(f'\n{customerID},{date},2pm,{guests}')
      elif timeSlot == 'evening':
        bookings.write(f'\n{customerID},{date},6pm,{guests}')
    extendedDisplay = VBox([sessionDisplay, Label(value='Booking successful.')])
  elif not validCustomer:
    extendedDisplay = VBox([sessionDisplay, Label(value='Invalid customerID.')])
  else:
    extendedDisplay = VBox([sessionDisplay, Label(value='Session is fully booked. Please choose another.')])
  if timeSlot == 'afternoon':
    currentChildren[0] = extendedDisplay
  elif timeSlot == 'evening':
    currentChildren[1] = extendedDisplay
  bookSessionDisplay.children = tuple(currentChildren)

## Inventory Management

In [ ]:
import datetime

'''
This blocks performs the necessary calculations needed for the graphs shown in
the 'Inventory' Tab.
- getBar returns an ordered list of the number of rentals/booked session for
every month in the last calendar year
- getPie returns the number of rentals made overall for each genre, alongside
the relevant labels and the average rating of each genre (while not currently
implemented, this is for future development as a display when hovering over a
particular genre on the pie chart)
'''

#Calculate values for Bar Chart
def getBar():
  year = datetime.datetime.now().year
  #Getting Rental Numbers
  rentals = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  with open("Rental.txt", "r") as rental:
    lines = rental.readlines()
  for line in lines[1:]:
    current = line.strip().split(',')
    if len(current) >= 2:
      try:
        date = datetime.datetime.strptime(current[1], '%Y-%m-%d')
        if date.year == year:
          index = date.month - 1
          rentals[index] += 1
      except ValueError:
        pass
  #Getting Booking Numbers
  bookings = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  with open("Bookings.txt", "r") as booking:
    lines = booking.readlines()
  for line in lines[1:]:
    current = line.strip().split(',')
    if len(current) >= 2:
      try:
        date = datetime.datetime.strptime(current[1], '%Y-%m-%d')
        if date.year == year:
          index = date.month - 1
          bookings[index] += 1
      except ValueError:
        pass
  return rentals, bookings

#Calculate values for Pie Chart
def getPie():
  gameGenres = {}
  # Board Games
  with open('BoardGameInfo.txt', 'r') as BoardGameInfo:
    for line in BoardGameInfo:
      if line.strip() and not line.startswith('GameID'):
        current = line.strip().split(',')
        if len(current) >= 4:
          gameID = current[0].strip()
          genre = current[3].strip()
          gameGenres[gameID] = genre

  # Video Games
  with open('VideoGameInfo.txt', 'r') as VideoGameInfo:
    for line in VideoGameInfo:
      if line.strip() and not line.startswith('GameID'):
        current = line.strip().split(',')
        if len(current) >= 4:
          gameID = current[0].strip()
          genre = current[3].strip()
          gameGenres[gameID] = genre

  genreRentalsCount = {}
  genreRatingsSum = {}
  genreRatingsCount = {}
  with open("Game_Feedback.txt", "r") as feedback:
    lines = feedback.readlines()
  for line in lines[1:]:
    current = line.strip().split(',')
    if len(current) >= 2:
      gameID = current[0].strip()
      try:
        rating = int(current[1].strip())
      except ValueError:
        continue

      if gameID in gameGenres:
        genre = gameGenres[gameID]
        genreRentalsCount[genre] = genreRentalsCount.get(genre, 0) + 1
        genreRatingsSum[genre] = genreRatingsSum.get(genre, 0) + rating
        genreRatingsCount[genre] = genreRatingsCount.get(genre, 0) + 1

  genreLabels = []
  genreSizes = []
  genreAvgRatings = {}

  for genre in genreRentalsCount:
    genreLabels.append(genre)
    genreSizes.append(genreRentalsCount[genre])
    if genreRatingsCount[genre] > 0:
      genreAvgRatings[genre] = genreRatingsSum[genre] / genreRatingsCount[genre]
    else:
      genreAvgRatings[genre] = 0
  return genreLabels, genreSizes, genreAvgRatings

## Main Program

In [ ]:
from ipywidgets import *
import matplotlib.pyplot as plt
import numpy as np

'''
The main program sets up the GUI for each of the program's functionalities
(Renting, Returning, Session Booking and Inventory). Each of these are then
added to a corresponding Tab to easily split up the menus for each task. The
overall output is managed using ipywidgets Output widget.
'''

#Find a Game
gameSearch = Text(
      placeholder = 'e.g. ABC123',
      description = 'GameID',
      disabled = False
  )
boardGameSearch = Button(description='Search')
boardGameSearch.on_click(lambda b: findGame(b, findGameDisplay, searchBoard, 'board', gameSearch.value))

videoGameSearch = Button(description='Search')
videoGameSearch.on_click(lambda b: findGame(b, findGameDisplay, searchVideo, 'video', gameSearch.value))

searchBoard = HBox([gameSearch, boardGameSearch])
searchVideo = HBox([gameSearch, videoGameSearch])

findGameDisplay = Accordion(
    children = [searchBoard, searchVideo]
)
findGameDisplay.set_title(0, 'Board Game')
findGameDisplay.set_title(1, 'Video Game')

#Return a Game
ratingInput = Text(
  placeholder = '1-10',
  description = 'Rating',
  disabled = False
)
feedbackInput = Textarea(
  placeholder = 'Enter game feedback here:',
  description = 'Feedback',
  disabled = False
)
returnButton = Button(description = 'Return Game')
returnButton.on_click(lambda b: returnGame(b, tab, returnGameDisplay, gameSearch.value, ratingInput.value, feedbackInput.value))
returnGameDisplay = VBox([gameSearch, ratingInput, feedbackInput, returnButton])

#Book a Session
date = DatePicker(
    description = 'Date',
    disabled = False
)
customerInput = Text(
    description = 'CustomerID',
    disabled = False
)
guestInput = Text(
    placeholder = '0-3',
    description = '# Guests',
    disabled = False
)
bookAfternoonButton = Button(description = 'Book')
bookAfternoonButton.on_click(lambda b: bookSession(b, bookSessionDisplay, bookAfternoon, 'afternoon', date.value, customerInput.value, int(guestInput.value)))
bookAfternoon = VBox([date, customerInput, guestInput, bookAfternoonButton])

bookEveningButton = Button(description = 'Book')
bookEveningButton.on_click(lambda b: bookSession(b, bookSessionDisplay, bookEvening, 'evening', date.value, customerInput.value, int(guestInput.value)))
bookEvening = VBox([date, customerInput, guestInput, bookEveningButton])

bookSessionDisplay = Accordion(
    children = [bookAfternoon, bookEvening]
)
bookSessionDisplay.set_title(0, 'Afternoon (2pm-6pm)')
bookSessionDisplay.set_title(1, 'Evening (6pm-10pm)')

#Inventory
#Bar Chart of Rentals/Bookings Over Time
barChart = Output()
with barChart:
  months = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
  rentals, bookings = getBar()
  activity = {
      'Rentals': rentals,
      'Bookings': bookings
  }
  x = np.arange(len(months))
  width = 0.25
  multiplier = 0
  fig, ax = plt.subplots(layout='constrained')
  for label, value in activity.items():
    offset = width * multiplier
    rects = ax.bar(x+offset, value, width, label=label)
    ax.bar_label(rects, padding=3)
    multiplier +=1
  ax.set_xticks(x + width/2, months)
  ax.legend(loc='upper left', ncols=2)
  plt.title("Yearly Rentals/Bookings")
  plt.show()

#Pie Chart of Genre Split in Rentals
pieChart = Output()
with pieChart:
    genreLabels, genreSizes, genreAvgRatings = getPie()
    fig, ax = plt.subplots()
    ax.pie(genreSizes, labels=genreLabels)
    plt.title("Genre Rentals")
    plt.show()
inventoryDisplay = HBox([barChart, pieChart])

tabTitles = ('Find a Game', 'Return a Game', 'Book a Session', 'Inventory')
tab = Tab(
    children = [findGameDisplay, returnGameDisplay, bookSessionDisplay, inventoryDisplay]
)
for i in range(len(tabTitles)):
  tab.set_title(i, tabTitles[i])
output = Output()
display(tab, output)